# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
!pip install category_encoders -q

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import time
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.compose import ColumnTransformer

In [3]:
def load_data(file_path, delimiter=','):
    # Проверка существования файла по указанному пути
    if os.path.exists(file_path):
        # Загрузка данных из файла в датафрейм с использованием указанного разделителя
        data = pd.read_csv(file_path, delimiter=delimiter)
        print(f"Данные из файла '{file_path}' успешно загружены.")
        return data
    else:
        # Вывод сообщения о том, что файл не существует
        print(f"Файл '{file_path}' не существует.")
        return None

# Замените путь к файлам на ваши
data = load_data('/datasets/autos.csv')

Данные из файла '/datasets/autos.csv' успешно загружены.


In [4]:
# Просмотр первых строк данных
display(data.head())

# Общая информация о данных
data.info()

# Описательная статистика
display(data.describe())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
# Удаление неинформативного признака NumberOfPictures
data = data.drop(['NumberOfPictures'], axis=1)

# Заполнение оставшихся пропущенных значений медианой для числовых признаков и модой для категориальных
for column in data.columns:
    if data[column].dtype == 'object':
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:
        data[column].fillna(data[column].median(), inplace=True)

# Фильтрация данных по разумным значениям
data = data[(data['RegistrationYear'] >= 1900) & (data['RegistrationYear'] <= 2024)]

display(data.head())
data.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no,2016-03-24 00:00:00,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
Int64Index: 354198 entries, 0 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354198 non-null  object
 1   Price              354198 non-null  int64 
 2   VehicleType        354198 non-null  object
 3   RegistrationYear   354198 non-null  int64 
 4   Gearbox            354198 non-null  object
 5   Power              354198 non-null  int64 
 6   Model              354198 non-null  object
 7   Kilometer          354198 non-null  int64 
 8   RegistrationMonth  354198 non-null  int64 
 9   FuelType           354198 non-null  object
 10  Brand              354198 non-null  object
 11  Repaired           354198 non-null  object
 12  DateCreated        354198 non-null  object
 13  PostalCode         354198 non-null  int64 
 14  LastSeen           354198 non-null  object
dtypes: int64(6), object(9)
memory usage: 43.2+ MB


In [6]:
data['FuelType'].unique()

array(['petrol', 'gasoline', 'lpg', 'other', 'hybrid', 'cng', 'electric'],
      dtype=object)

"gasoline" и "petrol" обозначают одно и то же топливо. В американском английском используется термин "gasoline", в то время как в британском английском предпочтительнее "petrol".

In [7]:
# Обновление категорий топлива
data.loc[:, 'FuelType'] = data['FuelType'].replace({'petrol': 'gasoline'})

# Проверка уникальных значений в категории FuelType
print(data['FuelType'].unique())

['gasoline' 'lpg' 'other' 'hybrid' 'cng' 'electric']


In [8]:
# Преобразование столбца 'DateCrawled' в формат datetime
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'])

# Извлечение года из столбца 'DateCrawled' и получение уникальных значений
data['DateCrawled'].dt.year.unique()

array([2016])

Выгрузка данных была в 2016 году.

In [9]:
pd.options.mode.chained_assignment = None
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

# Предположим, что целевая переменная называется 'Price'
target_column = 'Price'

# Разделение данных на тренировочную и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Преобразование дат в числовой формат
def convert_dates(df):
    df['DateCrawled'] = pd.to_datetime(df['DateCrawled']).astype(np.int64) / 10**9
    df['DateCreated'] = pd.to_datetime(df['DateCreated']).astype(np.int64) / 10**9
    df['LastSeen'] = pd.to_datetime(df['LastSeen']).astype(np.int64) / 10**9
    return df

# Применим преобразование дат
train_data = convert_dates(train_data)
test_data = convert_dates(test_data)

# Кодирование категориальных признаков с использованием TargetEncoder
target_encoder = TargetEncoder(cols=categorical_columns)

train_data_encoded = train_data.copy()
test_data_encoded = test_data.copy()

train_data_encoded[categorical_columns] = target_encoder.fit_transform(train_data[categorical_columns], train_data[target_column])
test_data_encoded[categorical_columns] = target_encoder.transform(test_data[categorical_columns])

# Определение X_train, y_train, X_test, y_test 
X_train = train_data_encoded.drop(columns=target_column)
y_train = train_data_encoded[target_column]

X_test = test_data_encoded.drop(columns=target_column)
y_test = test_data_encoded[target_column]

## Обучение моделей

In [10]:
# Функция для обучения модели и замера времени обучения
def train_model(model, X_train, y_train):
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    return model, training_time

# Функция для предсказания и замера времени предсказания
def predict_model(model, X_train):
    start_time = time.time()
    predictions = model.predict(X_train)
    prediction_time = time.time() - start_time
    return predictions, prediction_time

# Инициализация моделей с уменьшенными параметрами
models = {
    "RandomForest": RandomForestRegressor(n_estimators=10, random_state=42),  # Уменьшено количество деревьев
    "LightGBM": lgb.LGBMRegressor(n_estimators=10, random_state=42),  # Уменьшено количество итераций
    "LinearRegression": LinearRegression()
}

model_metrics = {}

# Определение RMSE как отрицательной метрики для cross_val_score
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Уменьшено количество прогонов в кросс-валидации
for name, model in models.items():
    # Время обучения
    trained_model, train_time = train_model(model, X_train, y_train)
    
    # Время предсказания
    _, predict_time = predict_model(trained_model, X_train)  # Используем тренировочную выборку для измерения времени
    
    # RMSE с использованием кросс-валидации
    rmse_cv = -cross_val_score(model, X_train, y_train, cv=3, scoring=rmse_scorer).mean()  # Уменьшено количество прогонов
    
    # Сохранение метрик модели
    model_metrics[name] = {
        "model": trained_model,
        "train_time": train_time,
        "predict_time": predict_time,
        "rmse_cv": rmse_cv
    }


## Анализ моделей

In [11]:
# Анализ метрик и выбор лучшей модели
best_model_name = min(model_metrics, key=lambda k: model_metrics[k]['rmse_cv'])
best_model_info = model_metrics[best_model_name]

print(f"Наилучшая модель: {best_model_name}")
print(f"Средний RMSE по кросс-валидации: {best_model_info['rmse_cv']}")
print(f"Время обучения: {best_model_info['train_time']}")
print(f"Время предсказания: {best_model_info['predict_time']}")

# Финальное тестирование на тестовой выборке
start_time = time.time()
best_model = best_model_info['model']
best_model.fit(X_train, y_train)  # Обучаем модель на всех обучающих данных
training_time_final = time.time() - start_time

start_time = time.time()
predictions = best_model.predict(X_test)
prediction_time_final = time.time() - start_time

rmse_final = mean_squared_error(y_test, predictions, squared=False)

print("Финальные результаты на тестовой выборке:")
print(f"RMSE: {rmse_final}")
print(f"Время обучения: {training_time_final}")
print(f"Время предсказания: {prediction_time_final}")

Наилучшая модель: LinearRegression
Средний RMSE по кросс-валидации: -3155.9567416575896
Время обучения: 0.27422022819519043
Время предсказания: 0.017877817153930664
Финальные результаты на тестовой выборке:
RMSE: 3172.4799536066816
Время обучения: 0.21182632446289062
Время предсказания: 0.08204770088195801


## Заключение

Исходя из критериев заказчика (качество предсказания, скорость предсказания, время обучения) LinearRegression представляется наилучшим выбором для данной задачи, поскольку она обеспечивает приемлемое качество предсказания при более высокой скорости обучения и предсказания, что соответствует критериям заказчика.